### DNA sequences: (1) GATGCGCAG, (2) GGCAGTA 

#### 1. The following code is given for the DNA sequence alignment using dynamic programming 

#### [Match = +2, Mismatch = -3, Gap = -1].

a. Find the bugs and complete the partial code (comment on your edits).

b. The code should print the scoring matrix.

c. The code should print all the optimal alignments with their scores. 

d. Also, print the best alignment obtained with its corresponding score.


In [6]:
import numpy as np
import pandas as pd

def nw(x, y, match=2, mismatch=-3, gap=-1):
    nx = len(x)
    ny = len(y)
    
    # Initialization of the matrix.
    F = np.zeros((nx + 1, ny + 1))
    F[:, 0] = np.linspace(0, gap * nx, nx + 1)
    F[0, :] = np.linspace(0, gap * ny, ny + 1)
    #In line 10, 11 -ve sign before gap is removed.

    # Pointers to trace through an optimal alignment.
    P = np.zeros((nx + 1, ny + 1), dtype=int)
    P[:, 0] = 3
    P[0, :] = 4

    # Matrix filling.
    for i in range(1, nx + 1):
        for j in range(1, ny + 1):
            if x[i-1] == y[j-1]:  # Match
                diag = F[i-1, j-1] + match
            else:  # Mismatch
                diag = F[i-1, j-1] + mismatch

            top = F[i-1, j] + gap
            left = F[i, j-1] + gap

            F[i, j] = max(diag, top, left)

            # Update the pointer matrix.
            if F[i, j] == diag:
                P[i, j] += 2  # Diagonal move
            if F[i, j] == top:
                P[i, j] += 3  # Up move
            if F[i, j] == left:
                P[i, j] += 4  # Left move
    #Matrix P is filled above to fill the pointers to trace through an optimal alignment.
    #We add 2 for diagonal move, add 3 if the score for that cell is coming from the above, add 4 if the score for that cell is coming from left cell.

    # Print scoring matrix using pandas DataFrame
    print("Scoring Matrix:")
    df = pd.DataFrame(F, index=['-'] + list(x), columns=['-'] + list(y))
    print(df)

    # Trace through an optimal alignment.
    i = nx
    j = ny
    optimal_alignments = []
    def func(i, j, first_seq, second_seq):
        if i > 0 or j > 0:
            if P[i, j] in [2, 5, 6, 9]:  # Diagonal move since for obtaining 2, 5, 6, 9 by adding at most 3 numbers from 2, 3, 4, one of them needs to be 2.
                func(i-1, j-1, x[i-1] + first_seq, y[j-1] + second_seq)
            if P[i, j] in [3, 5, 7, 9]:  # Up move since for obtaining 3, 5, 7, 9 by adding at most 3 numbers from 2, 3, 4, one of them needs to be 3.
                func(i-1, j, x[i-1] + first_seq, '-' + second_seq)
            if P[i, j] in [4, 6, 7, 9]:  # Left move since for obtaining 4, 6, 7, 9 by adding at most 3 numbers from 4, 6, 7, 9, one of them needs to be 4.
                func(i, j-1, '-' + first_seq, y[j-1] + second_seq)
        else:
            optimal_alignments.append((first_seq, second_seq))

    func(nx, ny, '', '')
    #Have replaced the while loop to a function to use recursion in order to find all possible optimal alignments.

    # Print all optimal alignments with their scores.
    for first_seq, second_seq in optimal_alignments:
        print ("Alignment:")
        print (first_seq)
        for e in range (len(first_seq)):
                if (first_seq[e]==second_seq[e]) :
                    print ("|", end="")
                else :
                    print (" ", end="")
        print()
        print (second_seq)
        print("Score: ", end="")
        print (F[i, j])
        print()
    
seq1 = "GATGCGCAG"
seq2 = "GGCAGTA" 

nw(seq1, seq2)
#removed print statement here becuase we are already printing inside the nw function.

Scoring Matrix:
     -    G    G    C    A    G    T    A
-  0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0
G -1.0  2.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0
A -2.0  1.0  0.0 -1.0  2.0  1.0  0.0 -1.0
T -3.0  0.0 -1.0 -2.0  1.0  0.0  3.0  2.0
G -4.0 -1.0  2.0  1.0  0.0  3.0  2.0  1.0
C -5.0 -2.0  1.0  4.0  3.0  2.0  1.0  0.0
G -6.0 -3.0  0.0  3.0  2.0  5.0  4.0  3.0
C -7.0 -4.0 -1.0  2.0  1.0  4.0  3.0  2.0
A -8.0 -5.0 -2.0  1.0  4.0  3.0  2.0  5.0
G -9.0 -6.0 -3.0  0.0  3.0  6.0  5.0  4.0
Alignment:
GATGC-G-CAG
|  || |  | 
G--GCAGT-A-
Score: 4.0

Alignment:
GATGC-GC-AG
|  || |  | 
G--GCAG-TA-
Score: 4.0

Alignment:
GATGCGCAG--
   | ||||  
---G-GCAGTA
Score: 4.0

Alignment:
GATGCGCAG--
|    ||||  
G----GCAGTA
Score: 4.0

Alignment:
GATGCGCAG--
|  |  |||  
G--G--CAGTA
Score: 4.0

Alignment:
GATGCGCAG--
|  ||  ||  
G--GC--AGTA
Score: 4.0



#### 2. Rectify the code for getting the most optimal local alignment (using dynamic programming) for the DNA sequences and the scoring scheme provided
#### [Match = +2, Mismatch = -1, Gap = -3].

a. Find the bugs and complete the partial code (comment on your edits).

b. The code should print the scoring matrix.

c. The code should print all the optimal alignments with their scores. 

d. Also, print the best alignment obtained with its corresponding score.


In [4]:
import numpy as np
import pandas as pd

def sw(x, y, match=2, mismatch=-1, gap=-3):
    nx = len(x)
    ny = len(y)
    
    # Initialization of the matrix.
    F = np.zeros((nx + 1, ny + 1))

    # Pointers to trace through an optimal alignment.
    P = np.zeros((nx + 1, ny + 1), dtype=int)
    P[:, 0] = 3
    P[0, :] = 4

    # Matrix filling.
    for i in range(1, nx + 1):
        for j in range(1, ny + 1):
            if x[i-1] == y[j-1]:  # Match
                diagonal = F[i-1, j-1] + match
            else:  # Mismatch
                diagonal = F[i-1, j-1] + mismatch

            
            top = F[i-1, j] + gap
            left = F[i, j-1] + gap

            F[i, j] = max(0, diagonal, top, left)  # This is the only difference from the NW algorithm

            # Update the pointer matrix.
            if F[i, j] == 0:
                P[i, j] = 0  # No move 
            if F[i, j] == diagonal:
                P[i, j] += 2  # Diagonal move
            if F[i, j] == top:
                P[i, j] += 3  # Up move
            if F[i, j] == left:
                P[i, j] += 4  # Left move
    #Matrix P is filled above to fill the pointers to trace through an optimal alignment.
    #We add 2 for diagonal move, add 3 if the score for that cell is coming from the above, add 4 if the score for that cell is coming from left cell, and 0 if there is no move.

    # Print scoring matrix using pandas DataFrame
    print("Scoring Matrix:")
    df = pd.DataFrame(F, index=['-'] + list(x), columns=['-'] + list(y))
    print(df)

    i=0
    j=0
    for a in range (10) :
        for b in range (8) :
            if (F[i][j]<F[a][b]) :
                i=a
                j=b

    optimal_alignments = []
    def func(i, j, first_seq, second_seq):
        if F[i, j] == 0:  # Stop traceback at score 0
            optimal_alignments.append((first_seq, second_seq))
        else:
            if P[i, j] in [2, 5, 6, 9]:  # # Diagonal move since for obtaining 2, 5, 6, 9 by adding at most 3 numbers from 2, 3, 4, one of them needs to be 2.
                func(i-1, j-1, x[i-1] + first_seq, y[j-1] + second_seq)
            if P[i, j] in [3, 5, 7, 9]:  # Up move since for obtaining 3, 5, 7, 9 by adding at most 3 numbers from 2, 3, 4, one of them needs to be 3.
                func(i-1, j, x[i-1] + first_seq, '-' + second_seq)
            if P[i, j] in [4, 6, 7, 9]:  # Left move since for obtaining 4, 6, 7, 9 by adding at most 3 numbers from 4, 6, 7, 9, one of them needs to be 4.
                func(i, j-1, '-' + first_seq, y[j-1] + second_seq)

    func(i, j, '', '')
    #Have replaced the while loop to a function to use recursion in order to find the local alignment.

    # Print all optimal alignments with their scores.
    for first_seq, second_seq in optimal_alignments:
        print ("Alignment:")
        print (first_seq)
        for e in range (len(first_seq)):
                if (first_seq[e]==second_seq[e]) :
                    print ("|", end="")
                else :
                    print (" ", end="")
        print()
        print (second_seq)
        print("Score: ", end="")
        print (F[i, j])

seq1 = "GATGCGCAG"
seq2 = "GGCAGTA" 

sw(seq1, seq2)
#removed print statement here becuase we are already printing inside the nw function.

Scoring Matrix:
     -    G    G    C    A    G    T    A
-  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
G  0.0  2.0  2.0  0.0  0.0  2.0  0.0  0.0
A  0.0  0.0  1.0  1.0  2.0  0.0  1.0  2.0
T  0.0  0.0  0.0  0.0  0.0  1.0  2.0  0.0
G  0.0  2.0  2.0  0.0  0.0  2.0  0.0  1.0
C  0.0  0.0  1.0  4.0  1.0  0.0  1.0  0.0
G  0.0  2.0  2.0  1.0  3.0  3.0  0.0  0.0
C  0.0  0.0  1.0  4.0  1.0  2.0  2.0  0.0
A  0.0  0.0  0.0  1.0  6.0  3.0  1.0  4.0
G  0.0  2.0  2.0  0.0  3.0  8.0  5.0  2.0
Alignment:
GCAG
||||
GCAG
Score: 8.0


all the possible alignments